In [1]:
!pip install autocorrect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.8/622.8 kB 15.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622363 sha256=1bb7baa6cc9d30025ccabf27998ff1c64d60ec29a9a2223ee1ef3b561b7f08f1
  Stored in directory: /root/.cache/pip/wheels/b5/7b/6d/b76b29ce11ff8e2521c8c7dd0e5bfee4fb1789d76193124343
Successfully built autocorrect


In [2]:
import pandas as pd
from autocorrect import Speller
from arrow import now

time_start = now()
checker = Speller(lang='en')

def score(arg: str, speller: Speller) -> float:
    result = 0
    result += 0.33 if len(arg) < 1798 else 0
    result += 0.33 if len(arg.split()) < 290 else 0
    result += 0.34 if arg == speller.autocorrect_sentence(sentence=arg) else 0
    return result

In [3]:
test_df = pd.read_csv(filepath_or_buffer='../input/llm-detect-ai-generated-text/test_essays.csv')
test_df['generated'] = test_df['text'].apply(func=score, args=(checker,))

In [4]:
test_df.drop(columns=['prompt_id', 'text']).to_csv('submission.csv')